In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import sys
import re

# sys.path.append(os.path.dirname(os.getcwd()))
# import mxd


In [2]:
def available_root_cause_descending1(all_layer_nodes, number,before_array,current_array,direction):
    all_nodes = []
#     result = []
    for layer in all_layer_nodes:
        for node in layer:
#             if node.request_number[1] == 0 or node.request_number[0] == 0:
#                 continue
            all_nodes.append(node)

    root_causes = []
    IIFs = []
    cps = []

    all_nodes.sort(key=lambda nod: nod.impact_factor, reverse=True)
    i = 0
    old = 1
    kkj = node.contribution_power
    for node in all_nodes:
        if kkj != old:
            i += 1
        old = kkj
        node.rank[0] = i
    all_nodes.sort(key=lambda nod:  nod.contribution_power, reverse=True)
    j = 0
    for node in all_nodes:
        j += 1
        node.rank[1] = j
    all_nodes.sort(key=lambda nod: sum(nod.rank)) 
    all_nodes = all_nodes[0:number]
    root_causes = []
    for node in all_nodes:
        element = node.path
        root_cause_dict = {}
        sign = 0
        for ele in element:
            root_cause_dict[ele[0]+1] = dim_list[ele[0]][ele[1]]
            if dim_list[ele[0]][ele[1]] == "NaN":
                sign=1
        if sign == 1:
            continue
        root_causes.append(root_cause_dict)
        IIF =  node.impact_factor  #差评比例
        cp =  node.contribution_power   #贡献度
        
        IIFs.append(IIF)
        cps.append(cp)

        print(root_cause_dict,"%.4f"%IIF,"%.4f"%cp)
    return root_causes,IIFs,cps

In [3]:
dff = pd.read_csv('./new_xsx.csv', encoding='utf-8-sig')

In [4]:
dff

,Unnamed: 0,年龄,性别,文化程度,目前社区是否存在集体产业？,人均年收入,社区产业是否依托了当地的自然地理与历史文化资源优势？,社区产业中是否包含了生态旅游等特色产业？,您认为目前社区产业发展最大的困难是什么？,基层党组织是否在社区产业发展中起到了引领作用？,基层党组织是否能够站在村民的立场上做事并为群众的利益服务？,社区目前的人口结构如何？,增加社区高质量人才的关键是？,当地社区内的年轻人是否愿意返乡创业？,社区现有的娱乐活动和文化活动是否能满足民众的精神需求？,社区在村史和乡村文化上是否做过宣传和研究工作？,保护村内传统文化设施对现代产业发展是起到了促进作用还是阻碍作用？,对社区目前居住环境的满意程度？,目前社区是否实现了经济与生态的协同发展？
0,0,3,1,0,0,2,0,0,3,0,0,2,2,0,0,0,2,4,0
1,1,3,0,4,1,2,0,0,3,0,0,0,2,1,1,0,2,3,0
2,2,2,1,3,0,0,0,1,3,0,0,2,2,0,1,1,0,4,1
3,3,2,0,0,0,3,0,1,2,0,0,0,0,0,1,1,0,6,1
4,4,2,1,3,1,2,1,1,0,1,0,2,1,0,0,0,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,2,0,3,0,4,0,1,0,0,0,0,2,1,1,1,0,5,1
85,85,1,0,0,0,3,0,1,0,0,0,2,0,1,1,1,0,3,1
86,86,3,1,4,0,1,0,1,2,0,0,0,2,1,1,1,2,2,1
87,87,3,1,4,0,3,0,1,0,0,0,0,0,1,1,1,0,2,1


In [5]:
df0 = dff.drop(['Unnamed: 0'],axis=1)

In [6]:
# %%time
useful_dim = []
names = []
guttman_coefficients = []
for index_name in df0.columns:   
    df = df0
    for name in df.columns:
        if name == index_name:
            continue
#         print("name:",name)
#         names.append(name)
        matrix = pd.crosstab(df[name], df[index_name])
    #     print(matrix)
        new = matrix.sum()
        matrix = matrix.append(new,ignore_index=True)
        temp = matrix.sum(axis=1)-matrix.max(axis=1)
        n = len(temp)-1
        if  n==0 or temp[n] == 0:
            guttman_coefficient = 100
        else:
            guttman_coefficient = (temp[n]-sum(temp[0:-1]))/temp[n]
        if guttman_coefficient > 0.2 :
#             useful_dim.append(name)
            useful_dim.append(index_name + " & " + name)
#             print(1)
        else:
            df =df.drop(name,axis= 1)
        if guttman_coefficient > 0.2 :
#             print(1)
#             print("name:",name)
            print(index_name + " & " + name)
            names.append(index_name + " & " + name)
            print("guttman_coefficient is:",guttman_coefficient)
            guttman_coefficients.append(guttman_coefficient)
            print("=======================================================")

人均年收入 & 目前社区是否存在集体产业？
guttman_coefficient is: 0.24489795918367346
人均年收入 & 社区产业中是否包含了生态旅游等特色产业？
guttman_coefficient is: 0.24489795918367346
人均年收入 & 当地社区内的年轻人是否愿意返乡创业？
guttman_coefficient is: 0.32653061224489793
人均年收入 & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？
guttman_coefficient is: 0.2857142857142857
人均年收入 & 社区在村史和乡村文化上是否做过宣传和研究工作？
guttman_coefficient is: 0.3469387755102041
人均年收入 & 保护村内传统文化设施对现代产业发展是起到了促进作用还是阻碍作用？
guttman_coefficient is: 0.32653061224489793
人均年收入 & 对社区目前居住环境的满意程度？
guttman_coefficient is: 0.20408163265306123
增加社区高质量人才的关键是？ & 对社区目前居住环境的满意程度？
guttman_coefficient is: 0.21428571428571427
当地社区内的年轻人是否愿意返乡创业？ & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？
guttman_coefficient is: 0.20833333333333334
社区现有的娱乐活动和文化活动是否能满足民众的精神需求？ & 社区在村史和乡村文化上是否做过宣传和研究工作？
guttman_coefficient is: 0.2608695652173913
社区在村史和乡村文化上是否做过宣传和研究工作？ & 基层党组织是否在社区产业发展中起到了引领作用？
guttman_coefficient is: 0.22727272727272727
社区在村史和乡村文化上是否做过宣传和研究工作？ & 基层党组织是否能够站在村民的立场上做事并为群众的利益服务？
guttman_coefficient is: 0.22727272727272727
社区在村史和乡村文化上是否做过宣传和研究工作

In [7]:
len(names)

13

In [8]:
len(guttman_coefficients)

13

In [9]:
pd.DataFrame({"name":names,"guttman_coefficient":guttman_coefficients})

,name,guttman_coefficient
0,人均年收入 & 目前社区是否存在集体产业？,0.244898
1,人均年收入 & 社区产业中是否包含了生态旅游等特色产业？,0.244898
2,人均年收入 & 当地社区内的年轻人是否愿意返乡创业？,0.326531
3,人均年收入 & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？,0.285714
4,人均年收入 & 社区在村史和乡村文化上是否做过宣传和研究工作？,0.346939
5,人均年收入 & 保护村内传统文化设施对现代产业发展是起到了促进作用还是阻碍作用？,0.326531
6,人均年收入 & 对社区目前居住环境的满意程度？,0.204082
7,增加社区高质量人才的关键是？ & 对社区目前居住环境的满意程度？,0.214286
8,当地社区内的年轻人是否愿意返乡创业？ & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？,0.208333
9,社区现有的娱乐活动和文化活动是否能满足民众的精神需求？ & 社区在村史和乡村文化上是否做过宣...,0.260870


In [10]:
useful_dim

['人均年收入 & 目前社区是否存在集体产业？',
 '人均年收入 & 社区产业中是否包含了生态旅游等特色产业？',
 '人均年收入 & 当地社区内的年轻人是否愿意返乡创业？',
 '人均年收入 & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？',
 '人均年收入 & 社区在村史和乡村文化上是否做过宣传和研究工作？',
 '人均年收入 & 保护村内传统文化设施对现代产业发展是起到了促进作用还是阻碍作用？',
 '人均年收入 & 对社区目前居住环境的满意程度？',
 '增加社区高质量人才的关键是？ & 对社区目前居住环境的满意程度？',
 '当地社区内的年轻人是否愿意返乡创业？ & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？',
 '社区现有的娱乐活动和文化活动是否能满足民众的精神需求？ & 社区在村史和乡村文化上是否做过宣传和研究工作？',
 '社区在村史和乡村文化上是否做过宣传和研究工作？ & 基层党组织是否在社区产业发展中起到了引领作用？',
 '社区在村史和乡村文化上是否做过宣传和研究工作？ & 基层党组织是否能够站在村民的立场上做事并为群众的利益服务？',
 '社区在村史和乡村文化上是否做过宣传和研究工作？ & 社区现有的娱乐活动和文化活动是否能满足民众的精神需求？']